In [2]:
import random
import numpy as np
import pandas as pd
import json
data=pd.read_csv("../data/decision_tree_data.csv",dtype="string_")
count_column=int(len(data)*(9/10))


In [3]:
# splitRandom function
# description : split data randomly 
def splitRandom(origin):
    list_all=np.arange(len(origin)).tolist()
    #randomly select amount of count in 0~ length of data(index of training) 
    list_training= random.sample(list_all, count_column)
    
    list_testing=[]#array store index of testing

    #test, training index are not duplicate

    for i in range(len(list_all)):
        for j in range(len(list_training)):
            if i==list_training[j]:
                break;
            else:
                if j==len(list_training)-1:
                    list_testing.append(i)
                
    
     #split data train, test using index
    data_training=origin.drop(list_testing)
    data_testing=origin.drop(list_training);

    return data_training, data_testing

In [4]:
def ID3(data, originaldata, features, target, parent_node_class=None):
    

    #Unique value of target data, number of each.
    elements, count=np.unique(data[target],return_counts=True)
   
  
    #attribute has single value (==child node is label data)
    if len(count)<=1:
       
        return elements[0]
   
    elif len(features)==0:
        return parent_node_class
    #bulid tree
    else:

        #select parent node
        parent_node_class=elements[np.argmax(count)]

        #store information gain of each attributes
        item_values=[]

        #print parent node entropy
      

        # store inforamtion gain of each features in itme_values
        for i in features:
            item_values.append(informationGain(data,i,target))
            
            

        #select the feature which is the biggest inforamation gain 
        index=np.argmax(item_values)
        select=features[index]

        
        #create tree structre
        tree={select:{}}

        #features store features except selected feature
        features=[i for i in features if i!=select]
    
        #draw child node of selected feature
        for value in np.unique(data[select]):
            sub_data=data.where(data[select]==value).dropna() #sub_data is data of selected features same with value
          
            subtree=ID3(sub_data, data, features, target, parent_node_class)#draw tree only using selected features
            tree[select][value]=subtree #store tree result
        return(tree)

In [5]:
def informationGain(data,column,target):
    entropy_total=entropy(data[target]) #calculate parent(root) entropy
    elements,counts=np.unique(data[column],return_counts=True) # elements is unique value , count is number of each value

    entropy_w=0 #entropy_w is weight entropy
    for i in range(len(elements)):
        en=entropy(data.where(data[column]==elements[i]).dropna()[target])#calculate entropy of each features
       
        entropy_w=entropy_w+counts[i]*en
        
    #calculate average
    entropy_w=entropy_w/np.sum(counts)
    

    #calcualte information gain and return it
    return entropy_total-entropy_w

In [ ]:
def entropy(data):
    elements,count=np.unique(data,return_counts=True)# elements is unique value , count is number of each value

    #if number of unqiue target value(labeld value) -> entropy is 0
    if len(count)==1:
        en=0
        
    # else calculate entropy
    else:
        p1=count[0]/np.sum(count)
        p2=count[1]/np.sum(count)
        en=-(p1*np.log2(p1)+p2*np.log2(p2))
    return en